In [1]:
import numpy as np
import pycolmap
import torch
import torch.nn as nn

from splat.gaussians import Gaussians
from splat.read_utils.read_gs_ply_files import convert_to_gaussian_schema, read_ply_file
from splat.utils import read_images_binary, read_images_text

# stem = "/Users/derek/Desktop/intro_to_gaussian_splatting"
stem = "/home/da2986/intro_to_gaussian_splatting"

# models_path = "/Users/derek/Downloads/models/"
models_path = "/home/da2986/intro_to_gaussian_splatting/models"
ply_path = f"/home/da2986/gaussian-splatting/truck/point_cloud/iteration_30000/point_cloud.ply"
vertices = read_ply_file(ply_path)
gaussians = convert_to_gaussian_schema(vertices)

colmap_path = f"{stem}/data/treehill/sparse/0"
colmap_path = f"/home/da2986/gaussian-splatting/tandt/truck/sparse/0"
reconstruction = pycolmap.Reconstruction(colmap_path)

points3d = reconstruction.points3D
images = read_images_binary(f"{colmap_path}/images.bin")
cameras = reconstruction.cameras

/home/da2986/intro_to_gaussian_splatting/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vertex Count: 2541226
Properties: [('float', 'x'), ('float', 'y'), ('float', 'z'), ('float', 'nx'), ('float', 'ny'), ('float', 'nz'), ('float', 'f_dc_0'), ('float', 'f_dc_1'), ('float', 'f_dc_2'), ('float', 'f_rest_0'), ('float', 'f_rest_1'), ('float', 'f_rest_2'), ('float', 'f_rest_3'), ('float', 'f_rest_4'), ('float', 'f_rest_5'), ('float', 'f_rest_6'), ('float', 'f_rest_7'), ('float', 'f_rest_8'), ('float', 'f_rest_9'), ('float', 'f_rest_10'), ('float', 'f_rest_11'), ('float', 'f_rest_12'), ('float', 'f_rest_13'), ('float', 'f_rest_14'), ('float', 'f_rest_15'), ('float', 'f_rest_16'), ('float', 'f_rest_17'), ('float', 'f_rest_18'), ('float', 'f_rest_19'), ('float', 'f_rest_20'), ('float', 'f_rest_21'), ('float', 'f_rest_22'), ('float', 'f_rest_23'), ('float', 'f_rest_24'), ('float', 'f_rest_25'), ('float', 'f_rest_26'), ('float', 'f_rest_27'), ('float', 'f_rest_28'), ('float', 'f_rest_29'), ('float', 'f_rest_30'), ('float', 'f_rest_31'), ('float', 'f_rest_32'), ('float', 'f_rest_33'

100%|██████████| 2541226/2541226 [00:24<00:00, 103463.73it/s]


[-0.37234583497047424, -0.41850927472114563, -0.25019344687461853]
tensor([0.3950, 0.3819, 0.4294])


In [2]:
cov_3D = gaussians.get_3d_covariance_matrix()

In [5]:
from torch import nn
import torch.nn.functional as F

def get_scaling_matrix(scales):
    return torch.exp(scales)

def build_rotation(r):
    norm = torch.sqrt(r[:,0]*r[:,0] + r[:,1]*r[:,1] + r[:,2]*r[:,2] + r[:,3]*r[:,3])

    q = r / norm[:, None]

    R = torch.zeros((q.size(0), 3, 3), device='cuda')

    r = q[:, 0]
    x = q[:, 1]
    y = q[:, 2]
    z = q[:, 3]

    R[:, 0, 0] = 1 - 2 * (y*y + z*z)
    R[:, 0, 1] = 2 * (x*y - r*z)
    R[:, 0, 2] = 2 * (x*z + r*y)
    R[:, 1, 0] = 2 * (x*y + r*z)
    R[:, 1, 1] = 1 - 2 * (x*x + z*z)
    R[:, 1, 2] = 2 * (y*z - r*x)
    R[:, 2, 0] = 2 * (x*z - r*y)
    R[:, 2, 1] = 2 * (y*z + r*x)
    R[:, 2, 2] = 1 - 2 * (x*x + y*y)
    return R

def get_3d_covariance_matrix(quaternions, scales) -> torch.Tensor:
    """
    Get the 3D covariance matrix from the scale and rotation matrix
    """
    # noramlize the quaternions
    quaternions = nn.functional.normalize(quaternions, p=2, dim=1)
    # nx3x3 matrix
    rotation_matrices = build_rotation(quaternions)
    # nx3x3 matrix
    scale_matrices = torch.zeros((len(quaternions), 3, 3)).to("cuda")
    scale = get_scaling_matrix(scales)
    scale_matrices[:, 0, 0] = scale[:, 0]
    scale_matrices[:, 1, 1] = scale[:, 1]
    scale_matrices[:, 2, 2] = scale[:, 2]
    m = rotation_matrices @ scale_matrices
    covariance = m @ m.transpose(1, 2)
    return covariance

scales = gaussians.scales.to("cuda")
quaternions = gaussians.quaternions.to("cuda")
cov_3D = get_3d_covariance_matrix(quaternions, scales)
print(cov_3D[:10])

tensor([[[ 1.8245e-06, -3.2099e-06,  1.6877e-05],
         [-3.2099e-06,  6.7294e-06, -3.0282e-05],
         [ 1.6877e-05, -3.0282e-05,  1.7309e-04]],

        [[ 6.7174e-05, -7.1285e-05, -4.1202e-06],
         [-7.1285e-05,  1.4332e-04,  2.3453e-05],
         [-4.1202e-06,  2.3453e-05,  5.6332e-06]],

        [[ 5.1018e-05, -7.2239e-06, -5.3563e-06],
         [-7.2239e-06,  1.0416e-04,  1.2372e-05],
         [-5.3563e-06,  1.2372e-05,  1.8711e-06]],

        [[ 1.9355e-04,  1.4669e-05, -6.0678e-05],
         [ 1.4669e-05,  3.4310e-04, -2.9783e-05],
         [-6.0678e-05, -2.9783e-05,  2.9225e-04]],

        [[ 1.3845e-04,  2.5536e-05, -1.0672e-04],
         [ 2.5536e-05,  4.8420e-06, -1.8272e-05],
         [-1.0672e-04, -1.8272e-05,  9.8527e-05]],

        [[ 1.0650e-02, -1.1133e-03, -2.0299e-04],
         [-1.1133e-03,  2.8228e-02, -3.3706e-02],
         [-2.0299e-04, -3.3706e-02,  4.0468e-02]],

        [[ 2.1503e-05, -1.2243e-04,  5.7590e-05],
         [-1.2243e-04,  1.3820e-03,  2

In [8]:
cov_3D[:10]

tensor([[[ 1.7807e-06, -3.1307e-06,  1.6432e-05],
         [-3.1307e-06,  6.5844e-06, -2.9479e-05],
         [ 1.6432e-05, -2.9479e-05,  1.6853e-04]],

        [[ 6.5680e-05, -6.9197e-05, -3.8869e-06],
         [-6.9197e-05,  1.3955e-04,  2.2888e-05],
         [-3.8869e-06,  2.2888e-05,  5.5292e-06]],

        [[ 5.0289e-05, -7.0303e-06, -5.2695e-06],
         [-7.0303e-06,  1.0205e-04,  1.2117e-05],
         [-5.2695e-06,  1.2117e-05,  1.8347e-06]],

        [[ 1.8810e-04,  1.3898e-05, -5.7923e-05],
         [ 1.3898e-05,  3.3069e-04, -2.8212e-05],
         [-5.7923e-05, -2.8212e-05,  2.8231e-04]],

        [[ 1.3442e-04,  2.4809e-05, -1.0344e-04],
         [ 2.4809e-05,  4.7097e-06, -1.7688e-05],
         [-1.0344e-04, -1.7688e-05,  9.5741e-05]],

        [[ 8.7432e-03, -8.3238e-04, -2.6451e-04],
         [-8.3238e-04,  1.7735e-02, -2.1157e-02],
         [-2.6451e-04, -2.1157e-02,  2.5421e-02]],

        [[ 2.0336e-05, -1.1413e-04,  5.5588e-05],
         [-1.1413e-04,  1.2834e-03,  1

In [4]:
from splat.image import GaussianImage
from splat.utils import (
    build_rotation,
    get_extrinsic_matrix,
    getIntinsicMatrix,
    read_camera_file,
    read_image_file,
)

camera_dict = read_camera_file(colmap_path)
image_dict = read_image_file(colmap_path)
images = {}
for idx in image_dict.keys():
    image = image_dict[idx]
    camera = camera_dict[image.camera_id]
    image = GaussianImage(camera=camera, image=image)
    images[idx] = image


image_num = 16
rotation_matrix = build_rotation(torch.Tensor(image_dict[image_num].qvec).unsqueeze(0))
translation = torch.Tensor(image_dict[image_num].tvec).unsqueeze(0)
extrinsic_matrix = get_extrinsic_matrix(rotation_matrix, translation).T
focal_x, focal_y = camera_dict[image_dict[image_num].camera_id].params[:2]
c_x, c_y = camera_dict[image_dict[image_num].camera_id].params[2:4]
width = camera_dict[image_dict[image_num].camera_id].width
height = camera_dict[image_dict[image_num].camera_id].height
intrinsic_matrix = getIntinsicMatrix(focal_x, focal_y, height, width).T

In [5]:
from splat.render_engine.gaussianScene2 import GaussianScene2

scene = GaussianScene2(gaussians=gaussians)
scene.device = "cuda"
TILE_SIZE = 16

processed_gaussians = scene.preprocess(
    extrinsic_matrix=extrinsic_matrix,
    intrinsic_matrix=intrinsic_matrix,
    focal_x=focal_x,
    focal_y=focal_y,
    width=width,
    height=height,
    tile_size=TILE_SIZE
)

torch.Size([2, 2541226]) torch.Size([2, 2541226])


In [7]:
import matplotlib.pyplot as plt

plt.scatter(
    processed_gaussians.means_3d[:, 0].detach().cpu(),
    processed_gaussians.means_3d[:, 1].detach().cpu(),
    c=processed_gaussians.color.detach().cpu(),
    s=1,
)

plt.xlim(0, width)
plt.ylim(0, height)
plt.gca().invert_yaxis()

Error in callback <function _draw_all_if_interactive at 0x7f5566a15c60> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [6]:
with torch.no_grad():
    output_image = scene.render_cuda(
        preprocessed_gaussians=processed_gaussians, height=height, width=width, tile_size=TILE_SIZE
    )

123 69


RuntimeError: CUDA out of memory. Tried to allocate 139.88 GiB (GPU 0; 14.57 GiB total capacity; 365.37 MiB already allocated; 13.23 GiB free; 478.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
from PIL import Image

image = output_image.cpu().numpy() * 255
image = Image.fromarray(image.astype(np.uint8))
image.save("test_output.jpg")

In [ ]:
from PIL import Image

image_path = "/Users/derek/Desktop/intro_to_gaussian_splatting/data/treehill/images/_DSC8973.JPG"
image = Image.open(image_path)

plt.imshow(image)
plt.show()